### Nonlinear Option Pricing (MATHGR5400), Columbia University, Spring 2020
# Homework II

### Due Date: 5:00 PM Friday, March 06, 2020
You should turn in the notebook at Columbia CourseWorks website

Please comment your code properly.

Before you turn in the notebook, press the "Run all cells" button in the toolbar, and make sure all the calculation results and graphs are produced correctly in a reasonable time frame, and then save the notebook.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# Dual Problem and Upper Bounds for American Options

<strong>Primal Problem</strong> (maximizing over stopping times)

\begin{equation*}
V_t=\sup_{\tau\in\mathcal{T}_{t, T}}\mathbb{E}^\mathbb{Q}\left[D_{t, \tau}F_{\tau}\big|\mathcal{F}_t\right]
\end{equation*}

<strong>Dual Problem</strong> (minimizing over martingales) [Rogers 2002; Haugh and Kogan 2004]

\begin{equation*}
V_t=\inf_{M\in\mathcal{M}_{t, 0}}\mathbb{E}^\mathbb{Q}\left[\sup_{t\leq s\leq T}\left(D_{t, s}F_s-M_s\right)\bigg|\mathcal{F}_t\right].
\end{equation*}

where $\mathcal{M}_{t, 0}$ denotes the set of all right-continuous martingales $(M_s, s\in[t, T])$ with $M_t=0$.
The optimal martingale $M^{\ast}$ is the martingale part of the Doob-Meyer decomposition of $(S_s-S_t)/D_{0, t}$, $t\leq s\leq T$, where $S$ is the Snell envelope of the discounted payoff, i.e., $S_t=\sup_{\tau\in\mathcal{T}_{t,T}}\mathbb{E}^{\mathbb{Q}}\left[D_{0,\tau}F_{\tau}\right]$.

In particular, any martingale $M_s$ with $M_0=0$ gives an upper bound for the price at time 0, $V_0$:

$$V_0 \leq\mathbb{E}^{\mathbb{Q}}\left[\sup_{0\leq s\leq T}\left(D_sF_s-M_s\right)\right].$$

<b style="color:darkorange">Question 1.</b> Consider pricing a one-year Bermudan put option with monthly exercise, where the asset price process is assumed to follow geometric Brownian motion with $S_0=100$, $\sigma=0.2$, $r=0.1$, $q=0.02$, $K=100$ and exercise dates $t_1=\frac{1}{12}$, $t_2=\frac{2}{12}$, $\cdots$, $t_{12}=1$.

Simulate 100,000 paths and use the following martingale to find a upper bound.

(a). $M_t\equiv0$.

(b). $M_t$ is the discounted European put price with the same final maturity less the initial price.

For your reference, the price of the option is $5.152$.

In [2]:
def blackscholes_mc(S=100, vol=0.2, r=0, q=0, ts=np.linspace(0, 1, 13), npaths=10):
    """Generate Monte-Carlo paths in Black-Scholes model.

    Parameters
    ----------
    S: scalar
        The spot price of the underlying security.
    vol: scalar
        The implied Black-Scholes volatility.
    r: scalar
        The annualized risk-free interest rate, continuously compounded.
    q: scalar
        The annualized continuous dividend yield.
    ts: array_like
        The time steps of the simualtion
    npaths: int
        the number of paths to simulate

    Returns
    -------
    paths: ndarray
        The Monte-Carlo paths.
    """
    nsteps = len(ts) - 1
    ts = np.asfarray(ts)[:, np.newaxis]
    W = np.cumsum(np.vstack((np.zeros((1, npaths), dtype=np.float),
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))),
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r-q)*ts)
    return paths


def blackscholes_price(K, T, S, vol, r=0, q=0, callput='call'):
    """Compute the call/put option price in the Black-Scholes model
    
    Parameters
    ----------
    K: scalar or array_like
        The strike of the option.
    T: scalar or array_like
        The maturity of the option, expressed in years (e.g. 0.25 for 3-month and 2 for 2 years)
    S: scalar or array_like
        The current price of the underlying asset.
    vol: scalar or array_like
        The implied Black-Scholes volatility.
    r: scalar or array_like
        The annualized risk-free interest rate, continuously compounded.
    q: scalar or array_like
        The annualized continuous dividend yield.
    callput: str
        Must be either 'call' or 'put'.

    Returns
    -------
    price: scalar or array_like
        The price of the option.

    Examples
    --------
    >>> blackscholes_price(95, 0.25, 100, 0.2, r=0.05, callput='put')
    1.5342604771222823
    """
    F = S*np.exp((r-q)*T)
    v = np.sqrt(vol**2*T)
    d1 = np.log(F/K)/v + 0.5*v
    d2 = d1 - v
    try:
        opttype = {'call':1, 'put':-1}[callput.lower()]
    except:
        raise ValueError('The value of callput must be either "call" or "put".')
    price = opttype*(F*norm.cdf(opttype*d1)-K*norm.cdf(opttype*d2))*np.exp(-r*T)
    return price

(a) $M_t = 0 $

\begin{equation*}
V_0=\mathbb{E}^\mathbb{Q}\left[\sup_{0\leq t\leq T}(D_{0, t}F_t)\right].
\end{equation*}

In [3]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
n_paths=100000
paths = blackscholes_mc(S0, vol, r, q, ts, n_paths)
payoff = np.maximum(K-paths[-1], 0) #payoff at maturity (under 10000 path) get a size 10000 array
V0 = payoff*np.exp(-r*(ts[-1]-ts[0]))

for i in range(len(ts)-1, 0, -1): #backwards, step-by-step discounting
    discount = np.exp(-r*(ts[i]-ts [0]))
    payoff = np.maximum(K-paths[i],0)*discount
    ind = payoff > V0
    V0[ind] = payoff[ind] #update for 'sup'
    
print("upperbound price (Mt=0):", np.mean(V0))

upperbound price (Mt=0): 8.608966860357073


Since setting $M_t=0$ is equivalent to make no hedge for the option, the price is the upper bound for fully covering risk exposure. Thus the price is very high. 

(b) $M_t$ is the discounted European put price with the same final maturity less the initial price.

$$V_0 =\mathbb{E}^{\mathbb{Q}}\left[\sup_{0\leq s\leq T}\left(D_sF_s-M_s\right)\right].$$

In [4]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
n_paths=100000
paths = blackscholes_mc(S0, vol, r, q, ts, n_paths)
payoff = np.maximum(K-paths[-1], 0)
M0 = blackscholes_price(K, T=ts[-1], S=paths[0], vol=0.2, r=0.1, q=0.02, callput='put') #initial value
Ms = payoff*np.exp(-r*(ts[-1]-ts[0])) #at T, Ms=payoff
V0 = payoff*np.exp(-r*(ts[-1]-ts[0]))-(Ms-M0)


for i in range(len(ts)-2, 0, -1): #backwards, step-by-step discounting
    discount = np.exp(-r*(ts[i]-ts[0]))
    payoff = np.maximum(K-paths[i],0)*discount
    Ms = blackscholes_price(K, T=ts[-1]-ts[i], S=paths[i], vol=0.2, r=0.1, q=0.02, callput='put')*discount
    payoff = payoff-(Ms-M0) #need to substract its initial value
    ind = payoff > V0  #find the maximum among all payoff
    V0[ind] = payoff[ind] #update for 'sup'
    
print("upperbound price (Mt = European put):", np.mean(V0))

upperbound price (Mt = European put): 5.363012259089903


## Optimal martingale

The optimal martingale that gives zero <em>duality gap</em> is the martingale component of the discounted true value process (known as Snell envelope) $\frac{V_t}{D_{0,t}}$, following the optimal exercise strategy.

To get a good upper bound, we first find a value process that is close to the true value process, and then extract its martingale component. This approximate value process can be obtained from a functional approximation such as regression, or from an exercise strategy (stopping time).

In general, for a stochastic process $\{U_n\}_{n\geq0}$, to extract its martingale component, we only need to let $M_0=0$, and then

$$M_{n+1}-M_n=U_{n+1}-\mathbb{E}\left[U_{n+1}\big|\mathcal{F}_n\right],\quad n=0,1,\cdots.$$

As a demonstration, we show below the code that calculates the upper bound estimate of the price of Bermudan put with monthly exercises as in Question 1.1. The optimal exercise frontier is given for each exercise date (from a PDE solver), i.e. if the underlying price falls below Bs[i] at time ts[i], then we should exercise the option, otherwise we should continue.

Using <b>Broadie-Andersen algorithm</b> to find the martingale and solve the dual problem.

In [23]:
#Optimal exercise value
Bs = np.array([87.5297, 87.7804, 88.0589, 88.3694, 88.724, 89.1309, 89.6067, 90.1723, 90.8653, 91.7451, 92.956, 94.8524, 100])

def exer_or_cont(i, S):
    return S <= Bs[i]

In [24]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
exer_func = lambda S: np.maximum(K-S, 0)

# nested simulation from time t_i when stock price is S
def nested_mc(S, vol, r, q, i, ts, nnested):   #return already discounted price
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        exer_vals =  exer_func(nested_paths)
        if j < len(ts)-1:
            ind = exer_or_cont(j, nested_paths)                    # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])  # sum all exercise payoff in each path, later take avg
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            if len(nested_paths) == 0:                             # if exercised for all paths, stop (might stop before T)
                break
        else: #final
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])      #final, exercise all left(max(0, payoff))
    return tot_payoff/nnested                                      # taking average of paths

# Simulate independent paths and exercise the option according to this strategy
V0 = nested_mc(S0, vol, r, q, 0, ts, 1000000)  #The primal is the similar as blackscholes_mc in Question 1.1, but using Bs as reference

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    exer_vals =  exer_func(paths[i])          
    ind = exer_or_cont(i, paths[i])                                # True for exercise False for continue
    for j in np.nonzero(ind)[0]: #exercised path
        V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    # if exercised, V[i,j] = exercise value
        EV[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
    for j in np.nonzero(~ind)[0]: #continued path
        V[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested) # if continue, use nested simulation to estimate V[i, j]
        EV[i, j] = V[i, j]                                         # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # M_{t_i} martinglae increment = V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))            #primal, simple forward mc guided by exer_or_cont
#find sup[D_{0,t}F_t-M_t] in each path and take avg of all paths
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1495
Dual Price   (Upper Bound) = 5.1569


Since the strategy is optimal (knowing optimal exercise price Bs as guidance), we see both lower bound and upper bound prices converge to the true value. 

However, in general, we may only get a sub-optimal strategy (e.g. from Longstaff-Schwartz algorithm). But with this sub-optimal strategy, we can build the (discounted) value process and then extract its martingale component to obtain an upper bound. Intuitively speaking, the closer this sub-optimal strategy is to the optimal one, the tighter the corresponding upper bound.

<b style="color:darkorange">Question 2.</b> Consider pricing the Bermudan put option as in Question 1.1 by using the constant $1.0$ and the Black-Scholes price of a European put option with volatility $0.2$ and maturity $T-t_n$ as the two basis functions at time $t_n$. Use the Longstaff-Schwartz algorithm to build an exercise strategy with these basis functions and then compute the corresponding upper bound.

For your convenience, an implementation of Longstaff-Schwartz algorithm is included in the cell below. It computes the regression coefficients at each exercise date.

In [7]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 10000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_LS = np.full((len(ts), 2), np.nan, dtype=np.float)
payoff = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_LS[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]    # regression to estimate continuation values
    contval = betas_LS[i, 0]+betas_LS[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]                     # update payoff on exercised paths

#Get the parameter for estimating cont-exer val in each time step
betas_LS                                           # No regression needed at first and last time steps, return NaN

array([[        nan,         nan],
       [-0.30380724,  1.27578698],
       [-0.41210061,  1.295443  ],
       [-0.2556016 ,  1.26075524],
       [-0.13401565,  1.22009766],
       [-0.1777664 ,  1.21977724],
       [-0.06304014,  1.16361705],
       [-0.04133896,  1.13067477],
       [-0.02901767,  1.10609109],
       [ 0.01335349,  1.06438054],
       [ 0.02016239,  1.03069796],
       [ 0.01094571,  0.9955471 ],
       [        nan,         nan]])

In [8]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
#define new cont-exer guidance using estimation from Longstaff-schwartz
def exer_or_cont(i, S):
    exerval = np.maximum(K-S, 0)
    Z = blackscholes_price(K, ts[-1]-ts[i], S, vol, r, q, callput='put')
    contval = betas_LS[i,0]+betas_LS[i,1]*Z
    return exerval > contval

In [9]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
exer_func = lambda S: np.maximum(K-S, 0)
# Simulate independent paths and exercise the option according to the longstaff-schwartz estimation
V0 = nested_mc(S0, vol, r, q, 0, ts, 1000000)  

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    exer_vals =  exer_func(paths[i])          
    ind = exer_or_cont(i, paths[i])                                # True for exercise False for continue
    for j in np.nonzero(ind)[0]: #exercised path
        V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    # if exercised, V[i,j] = exercise value
        EV[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
    for j in np.nonzero(~ind)[0]: #continued path
        V[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested) # if continue, use nested simulation to estimate V[i, j]
        EV[i, j] = V[i, j]                                         # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))            #primal, simple forward mc guided by exer_or_cont
#find sup[D_{0,t}F_t-M_t] in each path and take avg of all paths
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1453
Dual Price   (Upper Bound) = 5.1635


The duality gap is slightly larger now. Comparing the true value 5.152, we can see that the lower bound is less and the duality gap is larger than before, this is because the exercise guidance from longstaff-schwarts is a sub-optimal strategy. But in general this result is a good estimation, showing the sub-optimal strategy is pretty close the optimal one.

<b style="color:darkorange">Question 3.</b> <b>(An alternative algorithm to estimate the upper bound)</b> The primal algorithm (Longstaff-Schwartz or Tsitsiklis-Van Roy) provides a function approximation (via regression) of the continuation value $C_n$ at $t_n$, hence of the value function $V_n = \max(C_n,F_n)$ where $F_n$ is the exercise value. We can use this functional approximation of the value function to write an alternative algorithm in order to estimate the upper bound: at each time $t_n$, use the functional approximation of $V_n$, and run the nested simulation to estimate the conditional expectation $\mathbb{E}\left[V_{n+1}\big|\mathcal{F}_n\right]$ (using the functional approximation of $V_{n+1}$). Note that this nested simulation only needs to be run for <b>one time step</b>.

For the same Bermudan put option and same basis functions, use TVR algorithm to build the value process and then extract its martingale component using the alternative algorithm explained above to estimate the upper bound price.

For your convenience, an implementation of TVR algorithm is included in the cell below. It computes the regression coefficients at each exercise date. 

In [10]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 100000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_TVR = np.full((len(ts), 2), np.nan, dtype=np.float)
V = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_TVR[i] = np.linalg.lstsq(A, V*discount, rcond=None)[0]      # regression to estimate continuation values
    contval = betas_TVR[i, 0]+betas_TVR[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    V = np.maximum(exerval, contval)            #TVR update including the regression results, not only payoff

#Get the parameter for estimating cont-exer val in each time step
betas_TVR                                                 # No regression needed at first and last time steps, return NaN

array([[        nan,         nan],
       [-0.47191602,  1.32577963],
       [-0.37243706,  1.29210736],
       [-0.29563705,  1.26562192],
       [-0.20191512,  1.23048192],
       [-0.14999437,  1.20421299],
       [-0.07937057,  1.1689606 ],
       [-0.03328633,  1.13368039],
       [ 0.00295602,  1.09896827],
       [ 0.01297628,  1.06625151],
       [ 0.00145993,  1.03547019],
       [ 0.00652359,  1.00032146],
       [        nan,         nan]])

In [11]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
#define new cont-exer guidance using estimation from TVR
def exer_or_cont(i, S):
    exerval = np.maximum(K-S, 0)
    Z = blackscholes_price(K, ts[-1]-ts[i], S, vol, r, q, callput='put')
    contval = betas_TVR[i,0]+betas_TVR[i,1]*Z
    return exerval > contval

exer_func = lambda S: np.maximum(K-S, 0)

In [12]:
#define new nested_mc_altern according to alternative algorithm
def nested_mc_altern(S, vol, r, q, i, ts, nnested): #just one time-step
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    #for j in range(i+1, len(ts)):
    #only nested simulate one time-step
    dt = ts[i+1] - ts[i]
    dW = np.random.randn(len(nested_paths))*np.sqrt(dt) # Brownian increment
    nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)  #start from t_i, simulate for t_{i+1}
    if i < len(ts)-2:
        Z = blackscholes_price(K, ts[-1]-ts[i+1], nested_paths, vol, r, q, callput='put')
        cont_vals = betas_TVR[i+1,0]+betas_TVR[i+1,1]*Z
        exer_vals = exer_func(nested_paths)
        V_vals = np.maximum(cont_vals,exer_vals)           #use max(cont, exer) as V to update
    else: #i+1 is final, no betas_TVR (nan)
        exer_vals = exer_func(nested_paths)
        V_vals = exer_vals
    return np.mean(V_vals*np.exp(-r*ts[i+1])) 

In [13]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
exer_func = lambda S: np.maximum(K-S, 0)
# Simulate independent paths and exercise the option according to the longstaff-schwartz estimation
V0 = nested_mc(S0, vol, r, q, 0, ts, 1000000)  

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    cont_vals = betas_TVR[i,0]+betas_TVR[i,1]*Z
    exer_vals =  exer_func(paths[i])      
    ind = exer_vals > cont_vals                                    # True for exercise False for continue
    for j in np.nonzero(ind)[0]: #exercised path
        V[i, j] = np.maximum(exer_vals[j],cont_vals[j])*np.exp(-r*ts[i])  # if exercised, V[i,j] = max(exer, cont)
        EV[i,j] = nested_mc_altern(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
    for j in np.nonzero(~ind)[0]: #continued path
        #wrong
        # V[i,j] = nested_mc_altern(paths[i, j], vol, r, q, i, ts, nnested) # if continue, use nested simulation to estimate V[i, j]
        #right - In TVR, since we already know the cont_v, if contin, V=EV=max(cont_v,exer_v)=cont_v
        #(before we do not know so re-simulate to estimate)
        V[i, j] = cont_vals
        EV[i, j] = V[i, j]                                         # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))            #primal, simple forward mc guided by exer_or_cont
#find sup[D_{0,t}F_t-M_t] in each path and take avg of all paths
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1462
Dual Price   (Upper Bound) = 5.1542


The upper bound estimator can be used to check if the basis functions used in regression are adequate. A tight duality gap between the lower bound and upper bound is a confirmation that the linear span of the selected basis functions indeed gives a good approximation to the continuation value. We now illustrate this with the pricing of Bermudan-Asian call options.

<b style="color:darkorange">Question 4</b> (<b>Bermudan-Asian Call Option</b>). In Question 2.4 of Homework I, we priced a Bermudan-Asian call option using a particular set of basis functions. Assume that all the parameters stay the same.

For the Bermudan put option, the cashflow from exercising the option at time $t_n$, $n=1,\ldots,12$, is $\max\left(0, A_{t_n}-K\right)$ with $A_{t_n}=\frac{1}{n}\sum_{i=1}^nS_{t_i}$.

As for the basis functions in the regression at time $t_n$, use the constant 1.0 and the Black-Scholes value of a European call option with strike $K$, maturity $T-t_n$, and volatility $\bar\sigma = 0.1$, written on
$$
Z_{t_n}\equiv \frac{nA_{t_n}+(12-n)S_{t_n}}{12}
$$

<b>(a).</b> Reuse the exercise strategy derived from the basis functions used in Question 2.4 of Homework I by Longstaff-Schwartz with $\bar{\sigma}=0.1$ to estimate the lower and upper bound. For your convenience, an implementation of Longstaff-Schwartz is included in the cell below. Note that the lower bound should be obtained by running new independent paths stopped according to the strategy from the Longstaff-Schwartz algorithm.


In [14]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC = np.full((len(ts), 2), np.nan, dtype=np.float)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    A = np.vstack((np.ones_like(Z), callZ)).T
    betas_AC[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                                           # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
betas_AC

array([[        nan,         nan],
       [ 0.28574167,  1.13203731],
       [-0.01428628,  1.11233352],
       [-0.19211494,  1.09292166],
       [-0.30401035,  1.07595011],
       [-0.35797794,  1.06378967],
       [-0.39121996,  1.05050606],
       [-0.38731247,  1.04011448],
       [-0.36775062,  1.03394502],
       [-0.3170944 ,  1.02420177],
       [-0.25354092,  1.01731826],
       [-0.16828001,  1.00757378],
       [        nan,         nan]])

In [15]:
# new nested simulation for Bermudan-Asian Call Option
def nested_mc_AC(S, A, vol, r, q, i, ts, nnested):   #now need to pass in both St and At (avg price at time t, not all 0-t)
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]                                         # more than one-step for Bermuda-Asian option
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        A = A*(j-1)/j+nested_paths/j  #turn avg price at t to avg at t+1
        exer_vals = np.maximum(A-K,0)
        
        if j < len(ts)-1:
            Z = (j*A+(12-j)*nested_paths)/12
            #for j=len(ts-1), we can not calculate this, otherwise might have some term divided by 0 (warning)
            callZ = blackscholes_price(K, ts[-1]-ts[j], Z, vol_, callput='call') 
            cont_vals = betas_AC[j, 0]+betas_AC[j, 1]*callZ
            ind = exer_vals > cont_vals                           # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])  # sum all exercise payoff in each path, later take avg
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            A = A[~ind]
            if len(nested_paths) == 0:                             # if exercised for all paths, stop (might stop before T)
                break
        else: #final
            tot_payoff += np.sum(exer_vals)*np.exp(-r*ts[j])      #final, exercise all left(max(0, payoff))
            
    return tot_payoff/nnested                                      # taking average of paths

In [16]:
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
A0 = S0
ts = np.linspace(0, 1, 13)
# Simulate independent paths and exercise the option according to the longstaff-schwartz estimation
V0 = nested_mc_AC(S0, A0, vol, r, q, 0, ts, 1000000) 

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
pathsS = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
V = np.full(pathsS.shape, np.nan, dtype=np.float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(pathsS.shape, np.nan, dtype=np.float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value


for i in range(1, len(ts)-1):
    Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    cont_vals = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    exer_vals = np.maximum(pathsA[i]-K, 0)
    ind = exer_vals > cont_vals
    for j in np.nonzero(ind)[0]: #exercised path
        V[i,j] = exer_vals[j]*np.exp(-r*ts[i])                    # if exercised, V[i,j] = exercise value
        EV[i,j] = nested_mc_AC(pathsS[i, j], pathsA[i,j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
    for j in np.nonzero(~ind)[0]: #continued path
        V[i,j] = nested_mc_AC(pathsS[i, j], pathsA[i,j], vol, r, q, i, ts, nnested) # if continue, use nested simulation to estimate V[i, j]
        EV[i,j] = V[i,j]                                          # E[V_{i+1}|F_i] = V_i
V[-1] = np.maximum(pathsA[-1]-K,0)*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))            #primal, simple forward mc guided by exer_or_cont
#find sup[D_{0,t}F_t-M_t] in each path and take avg of all paths
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(np.maximum(pathsA[1:]-K,0)*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.3148
Dual Price   (Upper Bound) = 5.4168



<b>(b).</b> As an alternative set of basis functions at time $t_n$, use 
$$1, A_{t_n}, S_{t_n}, A_{t_n}^2, S_{t_n}^2, A_{t_n}S_{t_n}.$$
in the Longstaff-Schwartz algorithm and then give the corresponding lower bound and upper bound.


In [17]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC_altern = np.full((len(ts), 6), np.nan, dtype=np.float)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    A_matrix = np.vstack((np.ones_like(pathsA[i]), pathsA[i], pathsS[i], pathsA[i]**2, pathsS[i]**2, pathsA[i]*pathsS[i])).T
    betas_AC_altern[i] = np.linalg.lstsq(A_matrix, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = np.dot(betas_AC_altern[i],A_matrix.T)   # multiply each coeff with corresponding terms and add together
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
betas_AC_altern

array([[            nan,             nan,             nan,
                    nan,             nan,             nan],
       [ 1.57078973e+02, -1.81440968e+00, -1.81440968e+00,
         7.00634808e-03,  7.00634769e-03,  7.00634769e-03],
       [ 1.58941672e+02, -1.67774729e+00, -1.98584956e+00,
         9.49843438e-03,  1.24766881e-02, -8.42670803e-04],
       [ 1.50819344e+02, -1.91719647e+00, -1.57467090e+00,
         9.80842399e-03,  8.93213582e-03,  1.45355744e-03],
       [ 1.45555915e+02, -2.24769910e+00, -1.12853570e+00,
         1.35307451e-02,  8.31713712e-03, -2.31404423e-03],
       [ 1.41226737e+02, -2.39697205e+00, -8.85401040e-01,
         1.48751966e-02,  7.25066029e-03, -3.12303372e-03],
       [ 1.37254081e+02, -2.49054428e+00, -7.06605050e-01,
         1.51911246e-02,  5.80122600e-03, -2.46490203e-03],
       [ 1.31069325e+02, -2.52288727e+00, -5.42824998e-01,
         1.52942738e-02,  4.57129629e-03, -2.04829757e-03],
       [ 1.26861126e+02, -2.54593097e+00, -4.278

In [18]:
# new nested simulation for Bermudan-Asian Call Option
def nested_mc_AC(S, A, vol, r, q, i, ts, nnested):   #now need to pass in both St and At (avg price at time t, not all 0-t)
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]                                         # more than one-step for Bermuda-Asian option
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        A = A*(j-1)/j+nested_paths/j  #turn avg price at t to avg at t+1
        exer_vals = np.maximum(A-K,0)
        
        if j < len(ts)-1:
            #for j=len(ts-1), we can not calculate this, otherwise might have some term divided by 0 (warning)
            A_matrix = np.vstack((np.ones_like(nested_paths), A, nested_paths, A**2, nested_paths**2, A*nested_paths)).T
            cont_vals = np.dot(betas_AC_altern[j], A_matrix.T)
            ind = exer_vals > cont_vals                           # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])  # sum all exercise payoff in each path, later take avg
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            A = A[~ind]
            if len(nested_paths) == 0:                             # if exercised for all paths, stop (might stop before T)
                break
        else: #final
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])      #final, exercise all left(max(0, payoff))
            
    return tot_payoff/nnested                                      # taking average of paths

In [19]:
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
A0 = S0
ts = np.linspace(0, 1, 13)
# Simulate independent paths and exercise the option according to the longstaff-schwartz estimation
V0 = nested_mc_AC(S0, A0, vol, r, q, 0, ts, 1000000)

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
pathsS = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
V = np.full(pathsS.shape, np.nan, dtype=np.float)                  # discounted value process V_i (discounted to time zero)
EV = np.full(pathsS.shape, np.nan, dtype=np.float)                 # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value


for i in range(1, len(ts)-1):
    A_matrix = np.vstack((np.ones_like(pathsS[i]), pathsA[i], pathsS[i], pathsA[i]**2, pathsS[i]**2, pathsA[i]*pathsS[i])).T
    cont_vals = np.dot(betas_AC_altern[i], A_matrix.T)
    exer_vals = np.maximum(pathsA[i]-K, 0)
    ind = exer_vals > cont_vals
    for j in np.nonzero(ind)[0]: #exercised path
        V[i,j] = exer_vals[j]*np.exp(-r*ts[i])                    # if exercised, V[i,j] = exercise value
        EV[i,j] = nested_mc_AC(pathsS[i, j], pathsA[i,j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
    for j in np.nonzero(~ind)[0]: #continued path
        V[i,j] = nested_mc_AC(pathsS[i, j], pathsA[i,j], vol, r, q, i, ts, nnested) # if continue, use nested simulation to estimate V[i, j]
        V[i,j] = cont_vals
        EV[i,j] = V[i,j]                                          # E[V_{i+1}|F_i] = V_i
V[-1] = np.maximum(pathsA[-1]-K,0)*np.exp(-r*ts[-1])              # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                     # martinglae increment V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))           # primal, simple forward mc guided by exer_or_cont
#find sup[D_{0,t}F_t-M_t] in each path and take avg of all paths
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(np.maximum(pathsA[1:]-K,0)*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.0831
Dual Price   (Upper Bound) = 5.4030


<b>(c).</b> Compare the numerical duality gaps obtained from (a) and (b). Which set of basis functions does a better job at estimating the optimal exercise strategy?

We can see that the result from (b) is worse than (a): Its lower bound is smaller and the duality gap is larger. Since $A_t$ depends on $S_t$, the basis function in (b) has severe multi-collinearity and might lead to biased regression results. On contrary, the basis function in (a) has only 1 variable (if regarding 1 as coefficient) i.e. Black-Scholes model prices for European call, and it is highly related to the future payoff since we can always approximately replicate an exotic option using the vanilla European option. Therefore, the regression result from (a) is more reliable and thus yields a tighter estimation bound for the option price.